In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Concatenate,Dense, Subtract,Flatten,Lambda
import matplotlib.pyplot as plt
import cv2
from  scipy import ndimage

In [ ]:
!curl -O http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xzvf lfw.tgz

In [ ]:
def show(image, label):
    plt.figure()
    plt.imshow(image[0])
    # plt.title(labels[label])
    plt.title(label)
    plt.axis('off')

In [ ]:
import os
import random
BASE_PATH = 'lfw'

labels = [i for i in os.listdir(BASE_PATH)]
paths_with_more_than_2_images = []

train_labels = [i for i in labels if len(os.listdir(os.path.join(BASE_PATH,i)))>=2]
test_labels = list(set(labels)-set(train_labels))
print(len(train_labels),len(test_labels))